In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("../input/train_V2.csv")
train.head()

In [ ]:
train2 = pd.concat([train, pd.get_dummies(train['matchType'])], axis=1)
train2.drop(['Id', 'groupId', 'matchId', 'matchType'], axis=1, inplace=True)
train2.head()

In [ ]:
train2.fillna(0)
train2 = train2.dropna(axis=0)
X = train2.drop('winPlacePerc', axis=1)
y = train2['winPlacePerc']

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [ ]:
missing_val_count_by_column = (train2.isnull().sum())

In [ ]:
print(missing_val_count_by_column)

In [ ]:
reg.fit(X, y)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(LinearRegression(), X, y, scoring='neg_mean_absolute_error').mean()

Далее генерим фичи самым наитупейшим образом - поиском корреляций:

In [ ]:
cols_to_fit = [col for col in X.columns]
corr = train2[cols_to_fit].corr()

In [ ]:
corr[cols_to_fit[0]]

In [ ]:
newFeatures = []
for i in cols_to_fit:
    for j in cols_to_fit:
        if(i==j):
            continue
        newFeatures.append([corr[i][j], i, j])
        print(i+' '+j)

In [ ]:
newFeatures = sorted(newFeatures)

In [ ]:
newFeatures

In [ ]:
for i in range(50):
    print(i)
    X[newFeatures[i][1]+newFeatures[i][2]] = X[newFeatures[i][1]]*X[newFeatures[i][2]]

In [ ]:
cross_val_score(LinearRegression(), X, y, scoring='neg_mean_absolute_error').mean()

Как видим, точность увеличилась, можно было и дальше генерить таким образом фичи, но kaggle даёт только 17Гб оперативки, поэтому к увеличению числа новых фичей прибегать не будем